# Revista do Instituto de Ciências Econômicas, Administrativas e Contábeis – Sinergia

## Bibliotecas importadas

In [5]:
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/labfis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
plt.style.use('seaborn')

## Funções básicas

In [7]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [11]:
def listar_links_pagina(query, url_search, pg_num=1):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    div_results = soup.find_all('h2', {'class': 'title'})
    a_links = [a.find('a')['href'] for a in div_results]
    return a_links

In [12]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(query, url_search, pg_num)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [14]:
url_search_text = 'https://www.seer.furg.br/sinergia/search/search?query=__query__&searchJournal=4&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=&dateToMonth=&dateToDay=&dateToYear=&orderBy=score&orderDir=desc&searchPage=__pg_num_#results'
links = listar_links(url_search_text, 'contabilidade', pg_num=1)


SSLError: HTTPSConnectionPool(host='www.seer.furg.br', port=443): Max retries exceeded with url: /sinergia/search/search?query=contabilidade&searchJournal=4&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=&dateToMonth=&dateToDay=&dateToYear=&orderBy=score&orderDir=desc&searchPage=__pg_num_ (Caused by SSLError(CertificateError("hostname 'www.seer.furg.br' doesn't match either of '*.furg.br', 'furg.br'")))

In [17]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [50]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [49]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date}

In [51]:
def captura_resumo(soup):
    try:
        abstract = soup.find('div', {'class': 'item abstract'}).text
        abstract = abstract.replace('Resumo', '')
        abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [52]:
def captura_palavras_chave(soup):
    try:
        kw_tag = soup.find('div', {'class': 'item keywords'})
        kw = kw_tag.find('span', {'class': 'value'}).text.replace(',', '.').replace(';', '.')
        kw = ' '.join(kw.split())
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [47]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'item authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [45]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'item authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [24]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'periódico': 'Revista do Instituto de Ciências Econômicas, Administrativas e Contábeis – Sinergia'}
    url_search_text = 'https://www.seer.furg.br/sinergia/search/search?query=__query__&searchJournal=4&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=&dateToMonth=&dateToDay=&dateToYear=&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'url': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    #df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y-%d-%m')
    return df

In [25]:
link = links[0]
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')
captura_titulo(soup)

{'Título': '\n\t\tPráticas de Conformidade no Processo de Captação de Recursos das Organizações do Terceiro Setor\n\t'}

In [44]:
teste = '\n\t\tPráticas de Conformidade no Processo de Captação de Recursos das Organizações do Terceiro Setor\n\t'

' '.join(teste.split())



'Práticas de Conformidade no Processo de Captação de Recursos das Organizações do Terceiro Setor'

In [48]:
captura_autores(soup)

{'Qtd. de Autores': 5,
 'Autor 1': 'Ana Lúcia Fontes de S. Vasconcelos',
 'Autor 2': 'Albenici Correia de Melo',
 'Autor 3': 'Erika Borges Ferreira',
 'Autor 4': 'Valmor Slomski',
 'Autor 5': 'Vilma Slomski',
 'Autor 6': ''}

## Pesquisa por artigos

In [55]:
key_words_list = ['contabilidade ambiental', 'balanço social']
df = realizar_consulta(key_words_list)
df.head()

""


In [56]:
df.to_csv('../data/PCG.csv', index=False)

In [57]:
df

""
